In [9]:
import pandas as pd
from ddp_util import chatomid_to_url
import random
import csv

# import data + make lists

In [2]:
# load json file generated on the respective date by parsing a Monasterium db dump ("full20220125-1710") using /notebooks/XML2PD/XML2PD.ipynyb
all_df_full = pd.read_json("../../data/in/sampling/charters_2022-09-18-2211.json")
all_atomids = all_df_full.explode("atom_id")["atom_id"].rename("atomid").to_list()
all_atomids_with_tenor = (all_df_full[all_df_full["cei_tenor_joined"].astype(str) != ""])["atom_id"].explode("atom_id").to_list()

In [3]:
# load 1000 charter project sample, downloaded from GSheets
cv_atomids = pd.read_csv("../../data/in/sampling/1000charters_gsheet.csv", encoding="utf-8")
cv_atomids["atomid"] = cv_atomids["URL "].apply(lambda x : url_to_chatomid(x))
cv_atomids = cv_atomids["atomid"].to_list()

In [4]:
# load 1000 charter NLP sample, downloaded from Unicloud
nlp_atomids = pd.read_parquet("../../data/in/sampling/1000charters_nlp.parquet", engine="pyarrow")
nlp_atomids = (nlp_atomids.explode("atom_id")["atom_id"].reset_index(drop=True)).to_list()

# generate new nlp sample from charters with texts

In [7]:
random.seed(1337)
random.shuffle(all_atomids_with_text_less_cv_nlp)
nlp2_atomids = sorted(all_atomids_with_text_less_cv_nlp[:1000])

# make reduced lists

In [8]:
all_atomids_with_text_less_cv_nlp = list(sorted(set(all_atomids_with_tenor) - set(cv_atomids + nlp_atomids)))
all_atomids_less_cv_nlp_nlp2 = list(sorted(set(all_atomids) - set(cv_atomids + nlp_atomids + nlp2_atomids)))

# generate new 3k from all (minus 'used' atomids))

In [ ]:
assert len(set(nlp_atomids)) == len(set(cv_atomids)) == len(set(nlp2_atomids))
assert len(set(nlp_atomids + cv_atomids + nlp2_atomids)) == 3000

In [ ]:
random.seed(2448)
random.shuffle(all_atomids_less_cv_nlp_nlp2)
newsample_atomids = sorted(all_atomids_less_cv_nlp_nlp2[:3000])

# export

In [ ]:
with open("../../misc/1000_NLPCharters2/1k_urls.tsv","w") as f:
    w = csv.writer(f, delimiter="\t", dialect='unix', quoting=csv.QUOTE_NONE)
    for n, atomid in enumerate(nlp2_atomids):
        w.writerow([n,atomid,chatomid_to_url(atomid=atomid)])

In [ ]:
with open("../../misc/3000_rndCharters/3k_urls.tsv","w") as f:
    w = csv.writer(f, delimiter="\t", dialect='unix', quoting=csv.QUOTE_NONE)
    for n, atomid in enumerate(newsample_atomids):
        w.writerow([2000+n,atomid,chatomid_to_url(atomid=atomid)])

## all charters

In [12]:
with open("../../misc/all_charters/all_charters.tsv", "w") as f:
    w = csv.writer(f, delimiter="\t", dialect='unix', quoting=csv.QUOTE_NONE)
    for n, atomid in enumerate(all_atomids):
        try:
            w.writerow([n,atomid,chatomid_to_url(atomid=atomid)])
        except ValueError:
            continue

In [ ]:
# sample3k = pd.read_table("../../misc/3000_rndCharters/3k_urls.tsv")